# Проект по SQL
## Введение
Компания купила крупный сервис для чтения книг по подписке. Целью проекта является анализ базы данных. Для достижения поставленной цели необходимы выполнить следующие задачи:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

В распоряжении имеются следующие таблицы:
- Таблица books - содержит данные о книгах:
  - book_id — идентификатор книги;
  - author_id — идентификатор автора;
  - title — название книги;
  - num_pages — количество страниц;
  - publication_date — дата публикации книги;
  - publisher_id — идентификатор издателя.
- Таблица authors - содержит данные об авторах:
  - author_id — идентификатор автора;
  - author — имя автора.
- Таблица publishers - содержит данные об издательствах:
  - publisher_id — идентификатор издательства;
  - publisher — название издательства;
- Таблица ratings - содержит данные о пользовательских оценках книг:
  - rating_id — идентификатор оценки;
  - book_id — идентификатор книги;
  - username — имя пользователя, оставившего оценку;
  - rating — оценка книги.
- Таблица reviews - содержит данные о пользовательских обзорах на книги:
  - review_id — идентификатор обзора;
  - book_id — идентификатор книги;
  - username — имя пользователя, написавшего обзор;
  - text — текст обзора.

## Подключение к базе данных
Импортируем необходимые библиотеки и подключимся к базе данных. Тажке, выведем по 5 строк каждой таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
  '''Открываем соединение, получаем данные из sql, закрываем соединение'''
  with engine.connect() as con:
    return pd.read_sql(sql=sa.text(query), con = con)

# формируем запрос и выводим данные
query = '''SELECT * FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Выполнение заданий
### Количество книг после 1 января 2000 года

In [6]:
query = '''
SELECT COUNT(DISTINCT(book_id))
FROM books
WHERE publication_date::date >= '2000-01-01'
'''

get_sql_data(query)

,count
0,821


### Количество обзоров и средняя оценка для каждой книги

In [7]:
query = '''
WITH reviews_count AS
    (SELECT book_id,
            COUNT(DISTINCT(review_id)) AS reviews_count
    FROM reviews
    GROUP BY book_id),

avg_ratings AS
    (SELECT book_id,
            ROUND(AVG(rating), 1) AS avg_rating
    FROM ratings
    GROUP BY book_id)

SELECT b.book_id,
       b.title,
       rc.reviews_count,
       ar.avg_rating
FROM books AS b
LEFT OUTER JOIN reviews_count AS rc ON b.book_id = rc.book_id
LEFT OUTER JOIN avg_ratings AS ar ON b.book_id = ar.book_id

'''

get_sql_data(query)

,book_id,title,reviews_count,avg_rating
0,1,'Salem's Lot,2.0,3.7
1,2,1 000 Places to See Before You Die,1.0,2.5
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.7
3,4,1491: New Revelations of the Americas Before C...,2.0,4.5
4,5,1776,4.0,4.0
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.0
996,83,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.7
997,221,Essential Tales and Poems,NaN,4.0
998,387,Leonardo's Notebooks,NaN,4.0


### Издательство, выпустившее наибольшее число книг толщиной более 50 страниц

In [8]:
query = '''
WITH top_publisher_id AS
    (SELECT publisher_id,
            COUNT(DISTINCT(book_id))
     FROM books
     WHERE num_pages > 50
     GROUP BY publisher_id
     ORDER BY COUNT(DISTINCT(book_id)) DESC
     LIMIT 1)

SELECT publisher
FROM publishers AS p
RIGHT OUTER JOIN top_publisher_id AS t ON t.publisher_id = p.publisher_id

'''


get_sql_data(query)

,publisher
0,Penguin Books


### Автор книги с самой высокой средней оценкой, где количество оценок 50 и более

In [9]:
query = '''
WITH top_author_id AS
   (WITH table_1 AS 
        (SELECT b.author_id,
                b.book_id,
                AVG(rating) AS avg_rating
         FROM books AS b
         LEFT OUTER JOIN ratings AS r ON b.book_id = r.book_id
         GROUP BY b.author_id,
                  b.book_id
         HAVING COUNT(DISTINCT(rating_id)) >= 50)

    SELECT author_id,
           AVG(avg_rating)
    FROM table_1
    GROUP BY author_id
    ORDER BY AVG(avg_rating) DESC
    LIMIT 1)

SELECT a.author
FROM top_author_id AS tai
LEFT OUTER JOIN authors AS a ON a.author_id = tai.author_id
'''
get_sql_data(query)

,author
0,J.K. Rowling/Mary GrandPré


### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [10]:
query = '''
WITH table_2 AS
    (WITH table_1 AS
        (SELECT username,
                COUNT(DISTINCT(rating_id)) AS ratings_count
         FROM ratings
         GROUP BY username)

    SELECT t1.username,
           COUNT(DISTINCT(review_id)) AS reviews_count
    FROM table_1 AS t1
    LEFT OUTER JOIN reviews AS r ON r.username = t1.username
    WHERE t1.ratings_count > 48
    GROUP BY t1.username)

SELECT ROUND(AVG(reviews_count), 1)
FROM table_2

'''
get_sql_data(query)

,round
0,24.0


## Общие выводы
- Всего 821 книга, которые вышли после 1 января 2000 года;
- Для каждой книги посчитаны количество обзоров и средняя оценка;
- Издательство, которое выпустило наибольшее число книг толще
50 страниц — это Penguin Books;
- Автор с самой высокой средней оценкой книг, где количество оценок 50 и более - это J.K. Rowling/Mary GrandPré;
- Среднее количество обзоров от пользователей, которые поставили
больше 48 оценок, составляет 24.0 обзора.